# Data Cleaning – Credit Card Churn Dataset
This notebook performs initial data cleaning on the raw credit card churn dataset.  
The goal is to prepare the dataset for EDA and modeling by:
- Removing duplicates
- Handling missing values
- Addressing outliers
- Managing high-cardinality categorical features
- Handle Imbalanced Target Variable (`AttritionFlag`)
- Handle High Dimensionality
 
The cleaned dataset will be saved in `data/processed/` for use in later stages.

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid")

# Reproducibility
np.random.seed(42)

In [ ]:
# Paths
DATA_DIR = Path("../../data/raw")
FILE_PATH = DATA_DIR / "credit_card_attrition_dataset_mark.csv" 

In [ ]:
# Load
df = pd.read_csv(FILE_PATH)

## 1. Looking at the Dataset

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

*Remove unnecessary columns like `CustomerID`*

In [ ]:
# Drop CustomerID
df.drop(columns=["CustomerID"], inplace=True)

print("CustomerID column removed. New shape:", df.shape)

## 2. Checking for Duplicates

In [ ]:
# Count duplicates
df.duplicated().sum()

In [ ]:
# Remove duplicates
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

In [ ]:
# Get unique count for each variable
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
df.nunique()

In [ ]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [ ]:
df.shape

## 3. Checking for Missing Data

In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
print(df.isna().sum())

In [ ]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [ ]:
df[["Income", "CreditLimit", "TotalSpend"]].isnull().sum()

*The columns that has missing values area `Income`, `CreditLimit`, `TotalSpend` which has 5k missing  values.*

In [ ]:
cols_with_missing = ["Income", "CreditLimit", "TotalSpend"]

df[cols_with_missing].skew()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

figures_dir = Path("../../reports/figures")
figures_dir.mkdir(parents=True, exist_ok=True)

cols_with_missing = ["Income", "CreditLimit", "TotalSpend"]

for col in cols_with_missing:
    plt.figure(figsize=(6,4))
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f"Distribution of {col} (with Missing Values)")
    
    # Save figure
    save_path = figures_dir / f"{col}_distribution.png"
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    
    # Show plot
    plt.show()

    print(f"Saved: {save_path}")

*Since `Income` and `TotalSpend` were highly right-skewed, I will use median imputation to avoid distortion from outliers. `CreditLimit` was nearly symmetric, so I will use mean imputation to preserve its distribution.*

In [ ]:
# Median for skewed features
df["Income"] = df["Income"].fillna(df["Income"].median())
df["TotalSpend"] = df["TotalSpend"].fillna(df["TotalSpend"].median())

# Mean for symmetric feature
df["CreditLimit"] = df["CreditLimit"].fillna(df["CreditLimit"].mean())

In [ ]:
df[["Income", "CreditLimit", "TotalSpend"]].isnull().sum()

In [ ]:
df.describe()

## 4. Checking for Outliers

In [ ]:
# Get all numeric columns except target
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
if "AttritionFlag" in numeric_cols:
    numeric_cols.remove("AttritionFlag")

print("Numeric columns:", numeric_cols)

In [ ]:
# Function for detecting outliers using IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

figures_dir = Path("../../reports/figures/outliers_before")
figures_dir.mkdir(parents=True, exist_ok=True)

outlier_summary = []

for col in numeric_cols:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    outlier_summary.append({
        "Column": col,
        "Num_Outliers": len(outliers),
        "Lower_Bound": lower,
        "Upper_Bound": upper
    })
    
    # Save boxplot
    plt.figure(figsize=(6,4))
    sns.boxplot(x=df[col])
    plt.title(f"{col} - Outlier Detection")
    plt.savefig(figures_dir / f"{col}_boxplot.png", dpi=300, bbox_inches="tight")
    plt.close()

# Summary DataFrame
outlier_df = pd.DataFrame(outlier_summary).sort_values(by="Num_Outliers", ascending=False)
print(outlier_df)

In [ ]:
def cap_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df[column] = np.where(df[column] < lower_bound, lower_bound,
                          np.where(df[column] > upper_bound, upper_bound, df[column]))

# Apply to all numeric columns
for col in numeric_cols:
    cap_outliers_iqr(df, col)

print("Outliers capped for all numeric columns.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

figures_dir = Path("../../reports/figures/outliers_after")
figures_dir.mkdir(parents=True, exist_ok=True)

outlier_summary = []

for col in numeric_cols:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    outlier_summary.append({
        "Column": col,
        "Num_Outliers": len(outliers),
        "Lower_Bound": lower,
        "Upper_Bound": upper
    })
    
    # Save boxplot
    plt.figure(figsize=(6,4))
    sns.boxplot(x=df[col])
    plt.title(f"{col} - Outlier Detection")
    plt.savefig(figures_dir / f"{col}_boxplot.png", dpi=300, bbox_inches="tight")
    plt.close()

# Summary DataFrame
outlier_df = pd.DataFrame(outlier_summary).sort_values(by="Num_Outliers", ascending=False)
print(outlier_df)

*I detected outliers using the IQR method and capped them instead of removing them to preserve dataset size while reducing the influence of extreme values. This is particularly important for financial datasets where high values can be genuine but shouldn’t overly bias the model.*

## 5. Handling High Cardinality in Categorical Features

In [ ]:
# Identify categorical columns
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
print("Categorical columns:", categorical_cols)

In [ ]:
# Count unique values in each categorical column
for col in categorical_cols:
    print(f"{col}: {df[col].nunique()} unique values")

*The`Country` column is the only high cardinality, but we still need to convert categorical variables to numerical.*

In [ ]:
pd.set_option('display.max_rows', None) 

for col in categorical_cols:
    print(f"Distribution of {col}:")
    print(df[col].value_counts())
    print("-" * 40)

### *One hot encoding for low cardinality variables.*

In [ ]:
import pandas as pd

# Columns to one-hot encode
low_cardinality_cols = ["Gender", "MaritalStatus", "EducationLevel", "CardType"]

# One-hot encode
df_encoded = pd.get_dummies(df, columns=low_cardinality_cols, drop_first=False)

print("Shape before encoding:", df.shape)
print("Shape after encoding:", df_encoded.shape)

print("One-hot encoding done.")

*Note: `df_encoded` is the new df we are working with.*

In [ ]:
df_encoded.head(15)

### *Frequency Encoding for High Cardinality `Country` Column*

In [ ]:
# Frequency encoding for Country column
country_freq = df_encoded['Country'].value_counts(normalize=False)  # counts (not percentage)

# Map frequencies back to the dataframe
df_encoded['Country_FE'] = df_encoded['Country'].map(country_freq)

# Drop original Country column
df_encoded.drop(columns=['Country'], inplace=True)

print("Frequency encoding applied to Country column.")
print(df_encoded[['Country_FE']].head())

In [ ]:
df_encoded.shape

In [ ]:
df_encoded.head(15)